# Install packages

In [11]:
# %pip install lightgbm
# %pip install pandas-profiling
# %(pip install --upgrade pandas-profiling)
# %pip install ydata-profiling
%pip install pyarrow

                                              0.0/24.4 MB ? eta -:--:--
     -                                        0.7/24.4 MB 14.4 MB/s eta 0:00:02
     ---                                      2.2/24.4 MB 23.2 MB/s eta 0:00:01
     ------                                   3.9/24.4 MB 27.5 MB/s eta 0:00:01
     ---------                                5.9/24.4 MB 31.3 MB/s eta 0:00:01
     ------------                             7.9/24.4 MB 33.6 MB/s eta 0:00:01
     ----------------                         9.8/24.4 MB 36.6 MB/s eta 0:00:01
     ------------------                      11.7/24.4 MB 40.9 MB/s eta 0:00:01
     ----------------------                  13.9/24.4 MB 40.9 MB/s eta 0:00:01
     ------------------------                15.6/24.4 MB 43.5 MB/s eta 0:00:01
     ----------------------------            17.8/24.4 MB 40.9 MB/s eta 0:00:01
     -------------------------------         19.8/24.4 MB 38.5 MB/s eta 0:00:01
     ----------------------------------      21.

## Imports 

In [12]:
import pandas as pd
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import dateutil

## Load Sales Data


In [13]:
# ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
# A suitable version of pyarrow or fastparquet is required for parquet support.
# Trying to import the above resulted in these errors:
#  - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
#  - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [14]:
# Read Parquet 
sales_dataframe = pd.read_parquet("sales_orders_filtered (2).parquet")

# Keep the columns that are useful
sales_dataframe = sales_dataframe.drop(["order_status", "ft_sales_orders_identifier", 'division', 'facility', 'salesperson','warehouse', 'order_date','order_number', 'order_type', 'transaction_reason',
       'requested_delivery_date', 'requested_delivery_date_line','line_number',"highest_status_customer_order_head", "ordered_quantity_basic_u_m", "requested_delivery_date_obdwdt_", "invoiced_quantity_adjusted"], axis=1)

In [15]:
sales_dataframe

,customer_number,item_number,requested_delivery_date_obdwdz_,invoiced_quantity_basic_u_m
2,182080,11906175,2020-03-13,1500.0
3,182080,11906175,2020-03-13,1500.0
4,182080,11906175,2018-07-04,1500.0
5,182080,11906175,2018-07-04,1500.0
6,182080,11906175,2018-11-01,750.0
...,...,...,...,...
332961,158805,11863535,2021-04-14,70.0
332962,239177,11864685,2021-10-19,456.0
332963,239177,11864685,2021-10-19,456.0
332964,239177,11864685,2021-05-25,600.0


# Create time Series

Explainaition in here

In [16]:
# Get the min and max dates
min_date = sales_dataframe["requested_delivery_date_obdwdz_"].min()
max_date = sales_dataframe["requested_delivery_date_obdwdz_"].max()

# Create a date range with monthly frequency
date_range = pd.date_range(start=min_date, end=max_date, freq="M")

# Initialize an empty list to hold the monthly data
monthly_data = []

# Iterate through each customer
for customer in sales_dataframe["customer_number"].unique():
    customer_data = sales_dataframe[sales_dataframe["customer_number"] == customer]
    
    # Group the data by month and calculate the indicator and sum
    monthly_grouped = customer_data.groupby(pd.Grouper(key="requested_delivery_date_obdwdz_", freq="M")).agg(
        indicator_requested_delivery=("invoiced_quantity_basic_u_m", lambda x: int(len(x) > 0)),
        sum_of_invoiced_quantity_basic_u_m=("invoiced_quantity_basic_u_m", "sum")
    )
    
    # Reindex to the full date range and fill missing values
    monthly_grouped = monthly_grouped.reindex(date_range, fill_value=0)
    
    # Add customer number to each row
    monthly_grouped["customer_number"] = customer
    
    # Append to the monthly_data list
    monthly_data.append(monthly_grouped.reset_index())

# Concatenate the data for all customers
final_time_series_df = pd.concat(monthly_data, ignore_index=True)

print(final_time_series_df)

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [72]:
final_time_series_df







,index,indicator_requested_delivery,sum_of_invoiced_quantity_basic_u_m,customer_number
0,2018-02-28,0,0.0,182080
1,2018-03-31,0,0.0,182080
2,2018-04-30,0,0.0,182080
3,2018-05-31,0,0.0,182080
4,2018-06-30,0,0.0,182080
...,...,...,...,...
797227,2023-05-31,0,0.0,239177
797228,2023-06-30,0,0.0,239177
797229,2023-07-31,0,0.0,239177
797230,2023-08-31,0,0.0,239177


# Preprocessing 

## missing values 
We dont have any missing values in our dataset, but if we had this should have been the first step for preprocessing the time series 


## transform the dates to period
(not needed)

In [69]:
# sales_dataframe['requested_delivery_date_obdwdz_'] = pd.to_datetime(sales_dataframe['requested_delivery_date_obdwdz_']).dt.to_period('M')

## Groupby the sales to get the total volume per customer per order date

In [74]:
sales_dataframe = final_time_series_df.copy()

In [75]:
modelling_dataframe = sales_dataframe.groupby(['customer_number', "index"], as_index=False).sum()

# Feature engineering 


In [8]:
# max_date_df = modelling_dataframe.groupby('customer_number', as_index=False)['requested_delivery_date_obdwdz_'].max()
# max_date_df = max_date_df.rename(columns = {'requested_delivery_date_obdwdz_': "max_order_date"})
# merged_df = pd.merge(modelling_dataframe , max_date_df, on = "customer_number")

# # discover if the cusrtomers are active or not, e.g they made a purchase in the last 12 months
# # we create a new column, if the customer is active we place 1 if not 0 
# merged_df['cutoff_date'] = pd.Period('2022-03', 'M')
# merged_df['is_after_cutoff'] = merged_df['max_order_date'] > merged_df['cutoff_date']
# merged_df['is_after_cutoff'] = merged_df['is_after_cutoff'].astype(int)

# # drop columns that we dont need
# merged_df = merged_df.drop(["cutoff_date", "max_order_date"], axis=1)


# # drop not active customers because we are only predicting for active customers 
# merged_df = merged_df[merged_df['is_after_cutoff'] == 1]

# #drop the column after using it for filtering 
# merged_df = merged_df.drop('is_after_cutoff', axis=1)



## Area as a feature

In [76]:
area = pd.read_parquet("data_aggregated_per_customer_with_coords.parquet")
area = area[["customer_number", "area_categorical"]]

In [77]:
df = pd.merge(modelling_dataframe, area, on = 'customer_number', how = "left")

In [78]:
df = df.dropna()

## Lags as features

In [82]:
# one way
# for lag in range(2, 13):
#     df[f'lagged_volume{lag}'] = df['invoiced_quantity_basic_u_m'].shift(lag)
    
# another way to do it 
for lag in range(2, 13):
    df.loc[:, f'lagged_volume{lag}'] = df['sum_of_invoiced_quantity_basic_u_m'].shift(lag)

Using the  second piece of code that uses .loc for assignment is recommended. It not only avoids any potential issues but also makes the code more readable and adheres to the best practices of pandas DataFrame manipulation.

Clarity and Readability: Using .loc makes the code more explicit and self-explanatory. It clearly shows that you are using label-based indexing to assign values to specific rows and columns in the DataFrame.

Avoiding Warnings and Ambiguity: The first code snippet using direct assignment with square brackets and dot notation can trigger a warning about setting values on a copy of a slice from a DataFrame. While in this specific case it is unlikely to cause problems, it is best to use .loc to avoid any ambiguity and potential issues.

Best Practice: Using .loc for setting values in pandas DataFrames is considered a best practice because it explicitly indicates that you are modifying the original DataFrame and ensures code safety

# Rolling average

In [83]:
df_fe_rolling = df.drop(columns=['area_categorical', 'lagged_volume2',
       'lagged_volume3', 'lagged_volume4', 'lagged_volume5', 'lagged_volume6',
       'lagged_volume7', 'lagged_volume8', 'lagged_volume9', 'lagged_volume10',
       'lagged_volume11', 'lagged_volume12'])

In [84]:
dict_fe_rolling_agg = {
    "sum_of_invoiced_quantity_basic_u_m":"mean"
}

In [85]:
def get_rolling_avg(
    df,
    date_col,
    dict_features_to_roll,
    window
):
    """
    Creates rolling aggregation.
    """
    df_grouped = df.groupby(["customer_number",date_col],as_index= False).agg(dict_features_to_roll)
    for feature in dict_fe_rolling_agg:
        df_grouped[f"rolling_{window}_m_avg_{feature}"] = df_grouped[feature].rolling(window=window).mean()
        df_grouped[f"rolling_{window}_m_std_{feature}"] = df_grouped[feature].rolling(window=window).std()
        df_grouped.drop(columns=feature, axis=1, inplace=True)
        df_grouped.fillna(method="bfill", inplace=True)
    
    return df_grouped

In [17]:
# rolling averages
# how long they have been active? 
# DO WE WANT THEM STATIC OR ROLLIONG ???
# transformations scaling differences

In [87]:
df_fe_rolling

,customer_number,index,indicator_requested_delivery,sum_of_invoiced_quantity_basic_u_m
68,156859,2018-02-28,0,0.0
69,156859,2018-03-31,0,0.0
70,156859,2018-04-30,1,12600.0
71,156859,2018-05-31,0,0.0
72,156859,2018-06-30,0,0.0
...,...,...,...,...
751191,260392,2023-05-31,0,0.0
751192,260392,2023-06-30,0,0.0
751193,260392,2023-07-31,0,0.0
751194,260392,2023-08-31,0,0.0


In [88]:
rolling_average_df = get_rolling_avg(df_fe_rolling, "index", dict_fe_rolling_agg,3)

In [89]:
df_with_features = pd.merge(df, rolling_average_df, how='left', on=['customer_number','index' ])

## Transform categorical to numerical variables

In [90]:
# Convert categorical variables to numerical variables
df_with_features['area_categorical'] = pd.Categorical(df_with_features['area_categorical']).codes

# Split the dataset into input features and the target variable


In [93]:
df_with_features.columns

Index(['customer_number', 'index', 'indicator_requested_delivery',
       'sum_of_invoiced_quantity_basic_u_m', 'area_categorical',
       'lagged_volume2', 'lagged_volume3', 'lagged_volume4', 'lagged_volume5',
       'lagged_volume6', 'lagged_volume7', 'lagged_volume8', 'lagged_volume9',
       'lagged_volume10', 'lagged_volume11', 'lagged_volume12',
       'rolling_3_m_avg_sum_of_invoiced_quantity_basic_u_m',
       'rolling_3_m_std_sum_of_invoiced_quantity_basic_u_m'],
      dtype='object')

In [97]:
X = df_with_features.drop('indicator_requested_delivery', axis=1)
y = df_with_features['indicator_requested_delivery']

# Model 

In [ ]:
# lgbm


In [ ]:
df = df_with_features.copy()

In [101]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Preprocess the data
df['index'] = pd.to_datetime(df['index']).dt.to_period('M').apply(lambda r: pd.Timestamp(r.to_timestamp()))
df.sort_values('index', inplace=True)
df['index'] = pd.to_datetime(df['index']).dt.to_period('M').apply(lambda r: pd.Timestamp(r.to_timestamp()))

# Fill missing values in lagged volume features
lagged_features = ['lagged_volume2', 'lagged_volume3', 'lagged_volume4', 'lagged_volume5', 'lagged_volume6',
                   'lagged_volume7', 'lagged_volume8', 'lagged_volume9', 'lagged_volume10', 'lagged_volume11',
                   'lagged_volume12']
df[lagged_features] = df[lagged_features].fillna(0)

# Perform one-hot encoding for categorical columns
categorical_columns = ['area_categorical']
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Split the DataFrame into features (X) and target variable (y)
X = df_encoded.drop('indicator_requested_delivery', axis=1)
y = df_encoded['indicator_requested_delivery']

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Iterate over time series cross-validation splits
for train_index, test_index in tscv.split(X):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Convert the DataFrame into a LightGBM dataset format
    train_data = lgb.Dataset(X_train, label=y_train)

    # Define the LightGBM parameters
    params = {
        'objective': 'regression',
        'metric': 'mse',
        'num_leaves': 50,
        'learning_rate': 0.1,
        'feature_fraction': 0.9
    }

    # Train the LGBM model
    model = lgb.train(params, train_data, num_boost_round=100)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model using mean squared error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print('Mean Squared Error:', mse)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: customer_number, index

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Preprocess the data
df.reset_index(drop=True, inplace=True)  # Reset the index
df['requested_delivery_date_obdwdz__x'] = pd.to_datetime(df['requested_delivery_date_obdwdz__x']).dt.to_timestamp()
df.sort_values('requested_delivery_date_obdwdz__x', inplace=True)
df['requested_delivery_date_obdwdz__y'] = pd.to_datetime(df['requested_delivery_date_obdwdz__y']).dt.to_timestamp()

# Fill missing values in lagged volume features
lagged_features = ['lagged_volume2', 'lagged_volume3', 'lagged_volume4', 'lagged_volume5', 'lagged_volume6',
                   'lagged_volume7', 'lagged_volume8', 'lagged_volume9', 'lagged_volume10', 'lagged_volume11',
                   'lagged_volume12']
df[lagged_features] = df[lagged_features].fillna(0)

# Perform one-hot encoding for categorical columns
categorical_columns = ['area_categorical']
encoder = OneHotEncoder(sparse=False, drop='first')
encoded_cols = pd.DataFrame(encoder.fit_transform(df[categorical_columns]))
encoded_cols.columns = encoder.get_feature_names_out(categorical_columns)
df_encoded = pd.concat([df, encoded_cols], axis=1).drop(categorical_columns, axis=1)

# Convert the customer_number column to float
df_encoded['customer_number'] = pd.to_numeric(df_encoded['customer_number'], errors='coerce')

# Split the DataFrame into features (X) and target variable (y)
X = df_encoded.drop('invoiced_quantity_basic_u_m', axis=1)
y = df_encoded['invoiced_quantity_basic_u_m']

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Iterate over time series cross-validation splits
for train_index, test_index in tscv.split(X):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Convert the DataFrame into a LightGBM dataset format
    train_data = lgb.Dataset(X_train, label=y_train)

    # Define the LightGBM parameters
    params = {
        'objective': 'regression',
        'metric': 'mse',
        'num_leaves': 50,
        'learning_rate': 0.1,
        'feature_fraction': 0.9
    }

    # Train the LGBM model
    model = lgb.train(params, train_data, num_boost_round=100)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model using mean squared error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print('Mean Squared Error:', mse)
